![](1.jpg)

![](steps-min.png)

In [31]:
import pandas as pd 
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

---

In [33]:
df = sns.load_dataset('titanic')  
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


---

In [34]:
y = df.survived
X = df.drop(['survived',
             'embarked',
             'class',
             'who',
             'adult_male',
             'alive'], axis = 'columns')

In [38]:
X.head()

,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,female,35.0,1,0,53.1000,C,Southampton,False
4,3,male,35.0,0,0,8.0500,NaN,Southampton,True


In [39]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: survived, dtype: int64

In [40]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,
                                                y,
                                                test_size = .2,
                                                random_state = 0)

---

In [42]:
num_cols = Xtrain.select_dtypes(include = ['int64','float64']).columns
cat_cols = Xtrain.select_dtypes(include = 'object').columns 

---
- Automatisation du processus

In [26]:
def preprocessing(estimator = LogisticRegression()):
    num_pipeline = make_pipeline(
                             SimpleImputer(strategy = 'median'),
                             RobustScaler(),
                            )

    cat_pipeline = make_pipeline(
                     SimpleImputer(strategy = 'most_frequent'),
                     OneHotEncoder(handle_unknown = 'ignore', drop = 'if_binary')
                    )
    full_pipeline = make_column_transformer(
                               (num_pipeline, num_cols),
                               (cat_pipeline, cat_cols)
                             )  
    model = make_pipeline(full_pipeline, estimator)
    model.fit(Xtrain, ytrain)
    return model

In [46]:
model = preprocessing()

In [47]:
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  Index(['pclass', 'age', 'sibsp', 'parch', 'fare'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['sex', 'embark_town'], dtype='object'))])),
                ('logisticregression', LogisticRegression())])

---
- evaluer le modele avec la methode **score**

In [43]:
print('Le score obtenu est de:',round(model.score(Xtest,ytest), 2)*100,'%')

Le score obtenu est de: 80.0 %
